In [2]:
import json
import pprint
import pandas as pd
import csv
import os

In [6]:
def q4():
    years = [2016, 2017, 2018, 2019, 2020]
    game_types = ['regular_season', 'playoffs']
    columns = ['eventIdx', 'Date & Time', 'Period', 'Period Time', 'Period Time Remaining', 'Period Type', 
               'Game ID', 'Team Name', 'Home or Away', 'Shot or Goal', 'X-Coordinate', 'Y-Coordinate', 
               'Shooter Name', 'Goalie Name', 'Scorer Name', 'Shot Type', 'Was Net Empty', 
               'Goalie Strength', 'Home Rink Side']

    for year in years:
        for game_type in game_types:
            for file in os.listdir(f'../../JSON_data/{game_type}/{year}'):
                if file[-5:] == '.json':
                    # Opening JSON file
                    f = open(f'../../JSON_data/{game_type}/{year}/{file}')

                    game_id = int(file[:-5])

                    loaded_json = json.load(f)

                    all_plays = loaded_json['liveData']['plays']['allPlays']

                    data = []
                    
                    home_team = loaded_json['gameData']['teams']['home'].get('name')
                    
                    home_side = 'NA'
                    if len(loaded_json['liveData']['linescore']['periods']) > 0:
                        home_side = loaded_json['liveData']['linescore']['periods'][0]['home'].get('rinkSide')

                    for i in range(len(all_plays)):
                        event = all_plays[i]['result']['event']
                        if event == 'Shot' or event == 'Goal':
                            # pprint.pprint(all_plays[i]['result'])    
                            date_time = all_plays[i]['about']['dateTime']

                            period = all_plays[i]['about']['period']
                            period_time = all_plays[i]['about']['periodTime']
                            period_time_remaining = all_plays[i]['about']['periodTimeRemaining']
                            period_type = all_plays[i]['about']['periodType']

                            team_name = all_plays[i]['team']['name']
                            
                            if home_team == team_name:
                                home_or_away = 'Home'
                            else:
                                home_or_away = 'Away'

                            eventIdx = all_plays[i]['about']['eventIdx']

                            coord_x = 'NA'
                            coord_y = 'NA'
                            if 'coordinates' in all_plays[i]:
                                coord_x = all_plays[i]['coordinates'].get('x')
                                coord_y = all_plays[i]['coordinates'].get('y')

                            shooter_name = 'NA'
                            scorer_name = 'NA'
                            for player in all_plays[i]['players']:
                                if player['playerType'] == 'Shooter':
                                    shooter_name = player['player']['fullName']
                                elif player['playerType'] == 'Goalie':
                                    goalie_name = player['player']['fullName']
                                elif player['playerType'] == 'Scorer':
                                    scorer_name = player['player']['fullName']   

                            shot_type = 'NA'
                            if 'secondaryType' in all_plays[i]['result']:
                                shot_type = all_plays[i]['result']['secondaryType']

                            was_net_empty = False
                            goalie_strength = 'NA'
                            if event == 'Goal':
                                if 'emptyNet' in all_plays[i]['result']:
                                    was_net_empty = all_plays[i]['result']['emptyNet']

                                goalie_strength = all_plays[i]['result']['strength']['name']   

                            row_data = [eventIdx, date_time, period, period_time, period_time_remaining, 
                                        period_type, game_id, team_name, home_or_away, event, coord_x, coord_y, 
                                        shooter_name, goalie_name, scorer_name, shot_type, 
                                        was_net_empty, goalie_strength, home_side]

                            data.append(row_data)

                # create file path if it doesn't exist
                filename = f'../../CSV_data/{game_type}/{year}/{game_id}.csv'
                dirname = os.path.dirname(filename)
                if not os.path.exists(dirname):
                    os.makedirs(dirname)
                    
                # writing to csv file 
                with open(filename, 'w+') as csvfile: 
                    # creating a csv writer object 
                    csvwriter = csv.writer(csvfile) 

                    # writing the fields 
                    csvwriter.writerow(columns) 

                    # writing the data rows 
                    csvwriter.writerows(data)

In [7]:
q4()

# Question 4.2

We know that the strength of players on the ice starts off being even. So starting off, every event will be at even strength (5 on 5) until a penalty occurs. When a penalty occurs, the strength will change. The team whose player received a penalty will go to the penalty box leaving his team short handed and giving the other team a power play. This changes the strength of the players to 5 on 4. A penalty will last a certain number of minutes depending on the type of penalty. So, the strength of the players will change back to even automatically once the penalty expires, and we can check the game time of every event to know when to change the player strength back. Another possibility is if the team that has a power play scores a goal. Then, the strength will also change back to even.

# Question 4.3

In hockey, a rebound occurs when a puck gets shot and bounces off the frame of the goal post. We can speculate that shots or goals could have came from a rebound by paying attention to the time this event occurred and the previous event that occurred. If the previous event was a shot from the same team and happened within maybe 2 seconds, it is likely that the current shot came off of a rebound.

According to the youtube video linked in the instructions, a play off the rush appears to be when a single player is rushing towards the end he is trying to score with defenders trying to chase him to stop him. I don't know anything about hockey but I assume this happens when everyone is on one side of the rink and then possession changes all of a sudden (hence why there are no defenders on their goal side and they are chasing the player rushing). Possessions change when there is a giveaway or takeaway event. So, we can speculate that a players shot or goal came off the rush if the previous event was a giveaway or a takeaway and occurred within a few seconds.